In [11]:
#import all required libs not much are required so
import pandas as pd
import math
print('libs imported')

libs imported


In [12]:
def getdata():
    data = {
            'outlook':['sunny','sunny','overcast','rainy','rainy','rainy','overcast','sunny','sunny','rainy','sunny','overcast','overcast','rainy'],
            'temp':['hot','hot','hot','mild','cool','cool','cool','mild','cool','mild','mild','mild','hot','mild'],
            'humidity':['high','high','high','high','normal','normal','normal','high','normal','normal','normal','high','normal','high'],
            'windy':[False,True,False,False,False,True,True,False,False,False,True,True,False,True],
            'play':['no','no','yes','yes','yes','no','yes','no','yes','yes','yes','yes','yes','no']
    }
    data_frame = pd.DataFrame(data)
    return data_frame

def getBankData(split_fraction):
    if(split_fraction>1 and split_fraction<=0):
        print('Invalid Split Fraction')
        return None

    data_frame = pd.read_csv('bank.csv');
    #print(data_frame.head())
    data_frame = data_frame.sample(frac=1).reset_index(drop=True)
    #print(data_frame.head())
    mid = int(round(split_fraction*len(data_frame.index),0))
    # print(mid,len(data_frame.index))
    train_data = data_frame.iloc[ : mid , : ]
    test_data = data_frame.iloc[ mid : , :  ]

    # print(train_data.head())
    # print(test_data.head())

    return train_data,test_data


In [14]:
#test getdata and getBankData
data = getdata()
data.head()

,outlook,temp,humidity,windy,play
0,sunny,hot,high,False,no
1,sunny,hot,high,True,no
2,overcast,hot,high,False,yes
3,rainy,mild,high,False,yes
4,rainy,cool,normal,False,yes


In [16]:
#train_data
train_data,test_data=getBankData(0.5)
train_data.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
0,44,services,married,secondary,no,0,yes,yes,cellular,11,may,161,1,-1,0,unknown,no
1,66,retired,married,primary,no,206,no,no,cellular,9,feb,479,1,-1,0,unknown,yes
2,51,blue-collar,married,primary,no,21614,no,no,cellular,29,aug,47,20,-1,0,unknown,no
3,39,blue-collar,single,secondary,no,522,yes,no,telephone,14,may,225,7,350,2,failure,no
4,27,student,married,secondary,no,119,yes,no,cellular,16,jul,781,2,-1,0,unknown,yes


In [17]:
#test_data
test_data.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
5581,34,technician,married,secondary,no,260,yes,no,cellular,17,apr,20,2,238,2,failure,no
5582,42,technician,married,secondary,no,3082,no,no,cellular,11,sep,285,1,-1,0,unknown,yes
5583,32,admin.,single,secondary,no,223,yes,yes,unknown,27,may,236,4,-1,0,unknown,no
5584,48,admin.,married,secondary,no,191,yes,no,cellular,5,feb,238,3,-1,0,unknown,no
5585,42,blue-collar,married,secondary,no,443,yes,no,unknown,2,jun,671,5,-1,0,unknown,yes


In [18]:
def unique_vals(data,attr):
    """"find unique values for column in dataset"""
    col = data[attr].unique()
    return list(col)

def getClassCount(data,attr,classname):
    """counts no. of examples for each given class"""
    return list((data[attr].values)).count(classname)

def getProportion(data,attr,classname):
    """propotion of particular class value"""
    return getClassCount(data,attr,classname)/len(list((data[attr].values)))


def entropy(data,attr,classname,lcol):
    """
        this function returns entropy of particular
        atrribute : 'attr' with  labels stored in 'lcol'
        for particular class(value) as 'classname'
        --additional : pass attr=lcol to get entropy of system
    """
    pyes=0.0
    pno=0.0
    if attr == lcol:
        #this condition is to find the entropy of system
        pyes = getProportion(data,attr,'yes')
        pno =  getProportion(data,attr,'no')
        lgyes = 0.0
        lgno = 0.0
        if(pyes != 0.0):
            lgyes=math.log2(pyes)
        if(pno != 0.0):
            lgno=math.log2(pno)
        entrpy = -pyes*lgyes-pno*lgno;
        return entrpy
    #only consider a part of dataframe with value classname
    df = data[data[attr] == classname]
    n = len(list(df[attr].values))
    if(n==0):
        #no such attribute value the enntropy can not be determind
        print('Error : No such attribute as',classname)
        return -1.0
    pyes = len(list(df[df[lcol]=='yes'][attr].values))/n
    pno = len(list(df[df[lcol]=='no'][attr].values))/n
    
    lgyes = 0.0
    lgno= 0.0
    
    if(pyes != 0.0):
        lgyes=math.log2(pyes)
    if(pno != 0.0):
        lgno=math.log2(pno)
    #H(S) = -P1log2(P1)-P2log2(P2)
    entrpy = -pyes*lgyes-pno*lgno;
    return entrpy

def split(data,attr,classname):
    """split data for given classname of given attribute"""
    return data[data[attr]==classname]

def find_best_node(data):
    """
        returns the name and entropy of maximum entropy attribute for give 'data'
    """
    columns , lcol= list(data.columns)[:-1],list(data.columns)[-1]
    max_tot_entropy = -1.0
    best_attr = ""
    sys_entropy = entropy(data,lcol,'-',lcol);
    # print("{0:10}   {1:10}".format("attribute","entropy"))
    # print('-----------------------')  
    for col in columns:
        tot_entropy = 0.0
        for classname in unique_vals(data,col):
            tot_entropy += getProportion(data,col,classname)*entropy(data,col,classname,lcol)
        tot_entropy = sys_entropy-tot_entropy
        # print("{0:10}   {1:2f}".format(col,tot_entropy))
        if(max_tot_entropy<tot_entropy):
            max_tot_entropy = tot_entropy
            best_attr = col
    # print("**Best Choice :",best_attr)
    return best_attr,max_tot_entropy

In [35]:
print('unique vals outlook : ',unique_vals(getdata(),'outlook'))
print('number of elements with outlook as overcast are',getClassCount(getdata(),'outlook','overcast'))
print('proportion of overcast : ',getProportion(getdata(),'outlook','overcast'))
print("system's entropy :",entropy(getdata(),'play','temp','play'))
print('test split for sunny:')
print(split(getdata(),'outlook','sunny'))
print('H(sunny) : ',entropy(getdata(),'outlook','sunny','play'))
data = getdata()
print('(attr,gain) = ',find_best_node(data))

unique vals outlook :  ['sunny', 'overcast', 'rainy']
number of elements with outlook as overcast are 4
proportion of overcast :  0.2857142857142857
system's entropy : 0.9402859586706309
test split for sunny:
   outlook  temp humidity  windy play
0    sunny   hot     high  False   no
1    sunny   hot     high   True   no
7    sunny  mild     high  False   no
8    sunny  cool   normal  False  yes
10   sunny  mild   normal   True  yes
H(sunny) :  0.9709505944546686
(attr,gain) =  ('outlook', 0.2467498197744391)


In [36]:
class DecisionNode:
    """
        This Node Stores the attribute on which decision needs to be taken
        and further Sub-DecisionNodes
        data : dataframe of data on which this node was trained
        attr : decison attribute
        child : sub-decision Nodes 
    """
    def __init__(self,data,attr,child):
        self.data= data
        self.attr = attr
        self.childs = child


In [37]:
class Leaf:
    """
        Stores the prediction to return when its instance is encountered
    """
    def __init__(self,prediction):
        self.prediction  = prediction

In [38]:
class DecisionTree:

    """
        Decision Tree is the classifier class implementing ID3 Algorithm
        for decison making 
                                  
    """

    def __init__(self,data,lcol):
        """
            data :  pandas dataframe on which it will be trained
            lcol :  column in which labels are stored 
        """
        self.data=data
        self._root=None
        self.lcol = lcol
        pass
    
    def fit(self):
        """
            this is recursive fuction which trains 
                Decision tree and builds it simultaneously
        """
        self._root=self._fit(self.data,list(self.data.columns),self.lcol)

    def _fit(self,data,col_list,lcol):
        
        attr,gain = find_best_node(data)
        
        if(gain == 0 or len(unique_vals(data,lcol))==1 or len(col_list)==1):
            return Leaf(unique_vals(data,lcol)[0])
        
        branches = {}
        col_list.remove(attr)

        for classname in unique_vals(data,attr):
            next_data = split(data,attr,classname)
            branches[classname] = self._fit(next_data[col_list],col_list,lcol);
            
        return DecisionNode(data,attr,branches)
    
    def _tree_structure(self,root,lvl):
        if root == None:
            return
        if isinstance(root,DecisionNode):
            print('\t'*2*lvl,'attribute : ',root.attr)
        elif isinstance(root,Leaf):
            print('\t'*2*lvl,'prediction : ',root.prediction)
            return

        for classname in unique_vals(root.data,root.attr):
            try:
                print('\t'*2*lvl,'edge : ',classname)
                self._tree_structure(root.childs[classname],lvl+1)
            except : 
                pass

    def visualtize(self):
        """
            call this to visualize the tree structure
        """
        if self._root==None:
            print('Call fit() to create a tree first')
            return
        self._tree_structure(self._root,0)

    def predict_point(self,root,test_data_point):
        """
            predicts the label for single data_point
            and returns the result

            root : root of the decision tree
            test_data_point : data point for which result is to be predicted

        """
        prediction=''
        if(isinstance(root,Leaf)):
            prediction = root.prediction
        elif(isinstance(root,DecisionNode)):
            prediction=self.predict_point(root.childs[test_data_point[root.attr]],test_data_point)
        return prediction

    def predict(self,test_data,lcol):
        """
            test_data : data whose output needs to be predicted
            lcol      : column in which the lables are stored
            returns accurracy of predictions in precentage(%)
        """
        if(list(self.data.columns)!=list(test_data.columns)):
            #data passed to the for prediction is not made for this decison tree
            print('Error : Data Columns does not match')
            print('required :',list(self.data.columns))
            print('found :',list(test_data.columns))
            return None
        cols = list(test_data.columns)
        cols.remove(lcol)
        test_records = test_data[cols].to_dict(orient = 'records')
        test_labels = test_data[[lcol]].to_dict(orient = 'records')
        currect_cnt = 0
        tot_records = len(test_records)
        rec_no = 0 
        for record in test_records:
            try : 
                prediction = self.predict_point(self._root,record)
                if prediction == test_labels[rec_no][lcol]:
                    currect_cnt+=1
            except  KeyError:
                pass
            rec_no+=1
        
        return (currect_cnt/tot_records)*100

    def getRoot(self):
        return self._root; 

In [41]:
#test for wheather data set
data = getdata()
test_data=data
tree = DecisionTree(data,list(data.columns)[-1])
tree.fit()
tree.visualtize()
test_point = {
    'outlook':'rainy',
    'temp' : 'hot',
    'humidity' : 'high',
    'windy' : False,
}
print('data point prediction for',test_point,'is',tree.predict_point(tree.getRoot(),test_point))
print('Accurracy : ',tree.predict(test_data,list(test_data.columns)[-1]),'for given data')

 attribute :  outlook
 edge :  sunny
		 attribute :  humidity
		 edge :  high
				 prediction :  no
		 edge :  normal
				 prediction :  yes
 edge :  overcast
		 prediction :  yes
 edge :  rainy
		 attribute :  windy
		 edge :  False
				 prediction :  yes
		 edge :  True
				 prediction :  no
data point prediction for {'outlook': 'rainy', 'temp': 'hot', 'humidity': 'high', 'windy': False} is yes
Accurracy :  100.0 for given data


In [44]:
#test for bank data set
data,test_data = getBankData(0.75)
tree = DecisionTree(data,list(data.columns)[-1])
tree.fit()
tree.visualtize()
print('Accurracy : ',tree.predict(test_data,list(test_data.columns)[-1]))

 attribute :  balance
 edge :  3528
		 prediction :  yes
 edge :  333
		 attribute :  age
		 edge :  25
				 prediction :  yes
		 edge :  46
				 prediction :  no
		 edge :  63
				 prediction :  no
		 edge :  33
				 prediction :  no
		 edge :  57
				 prediction :  no
		 edge :  32
				 prediction :  yes
 edge :  680
		 prediction :  no
 edge :  1027
		 attribute :  duration
		 edge :  177
				 prediction :  yes
		 edge :  59
				 prediction :  no
		 edge :  313
				 prediction :  yes
		 edge :  744
				 prediction :  yes
		 edge :  151
				 prediction :  no
		 edge :  217
				 prediction :  yes
 edge :  3756
		 prediction :  no
 edge :  382
		 attribute :  job
		 edge :  services
				 prediction :  no
		 edge :  student
				 prediction :  yes
		 edge :  management
				 prediction :  no
		 edge :  technician
				 attribute :  day
				 edge :  13
						 prediction :  no
				 edge :  5
						 prediction :  yes
		 edge :  admin.
				 prediction :  no
 edge :  3
		 attribute :  campaign


		 prediction :  yes
 edge :  7968
		 prediction :  yes
 edge :  4112
		 prediction :  yes
 edge :  917
		 prediction :  no
 edge :  4693
		 prediction :  yes
 edge :  2845
		 prediction :  yes
 edge :  7585
		 prediction :  yes
 edge :  3730
		 prediction :  no
 edge :  951
		 prediction :  no
 edge :  622
		 prediction :  no
 edge :  1539
		 prediction :  yes
 edge :  -1042
		 prediction :  yes
 edge :  140
		 prediction :  no
 edge :  893
		 prediction :  no
 edge :  2228
		 prediction :  yes
 edge :  57
		 prediction :  yes
 edge :  5691
		 prediction :  no
 edge :  6574
		 prediction :  yes
 edge :  91
		 prediction :  yes
 edge :  975
		 prediction :  no
 edge :  70
		 prediction :  yes
 edge :  -157
		 prediction :  no
 edge :  476
		 prediction :  no
 edge :  5261
		 prediction :  no
 edge :  -17
		 prediction :  yes
 edge :  1075
		 prediction :  yes
 edge :  1311
		 prediction :  yes
 edge :  6798
		 prediction :  no
 edge :  482
		 prediction :  yes
 edge :  411
		 predictio

 edge :  4874
		 prediction :  no
 edge :  5205
		 prediction :  no
 edge :  928
		 prediction :  no
 edge :  7506
		 prediction :  yes
 edge :  1755
		 prediction :  no
 edge :  8729
		 prediction :  yes
 edge :  6402
		 prediction :  yes
 edge :  5169
		 prediction :  yes
 edge :  496
		 prediction :  yes
 edge :  111
		 prediction :  yes
 edge :  2467
		 prediction :  yes
 edge :  894
		 prediction :  no
 edge :  392
		 prediction :  no
 edge :  -295
		 prediction :  no
 edge :  395
		 prediction :  yes
 edge :  3176
		 prediction :  yes
 edge :  229
		 prediction :  no
 edge :  289
		 prediction :  yes
 edge :  6839
		 prediction :  no
 edge :  8929
		 prediction :  yes
 edge :  3629
		 prediction :  no
 edge :  5090
		 prediction :  yes
 edge :  -194
		 prediction :  no
 edge :  1649
		 prediction :  yes
 edge :  354
		 prediction :  no
 edge :  1412
		 prediction :  no
 edge :  3951
		 prediction :  yes
 edge :  27696
		 prediction :  yes
 edge :  1384
		 prediction :  no
 edge :

 edge :  2066
		 prediction :  yes
 edge :  -666
		 prediction :  no
 edge :  4660
		 prediction :  no
 edge :  1988
		 prediction :  yes
 edge :  16992
		 prediction :  no
 edge :  1516
		 prediction :  no
 edge :  2047
		 prediction :  no
 edge :  1780
		 prediction :  no
 edge :  -880
		 prediction :  no
 edge :  527
		 prediction :  yes
 edge :  585
		 prediction :  no
 edge :  4299
		 prediction :  no
 edge :  3047
		 prediction :  yes
 edge :  -311
		 prediction :  no
 edge :  6392
		 prediction :  no
 edge :  441
		 prediction :  no
 edge :  -139
		 prediction :  no
 edge :  1954
		 prediction :  no
 edge :  2030
		 prediction :  no
 edge :  20772
		 prediction :  no
 edge :  2994
		 prediction :  no
 edge :  3572
		 prediction :  no
 edge :  1190
		 prediction :  yes
 edge :  3434
		 prediction :  yes
 edge :  2308
		 prediction :  no
 edge :  904
		 prediction :  no
 edge :  6200
		 prediction :  no
 edge :  3773
		 prediction :  no
 edge :  1717
		 prediction :  no
 edge :  4

		 prediction :  no
 edge :  746
		 prediction :  no
 edge :  -498
		 prediction :  no
 edge :  2708
		 prediction :  no
 edge :  3856
		 prediction :  yes
 edge :  13094
		 prediction :  yes
 edge :  1604
		 prediction :  yes
 edge :  22086
		 prediction :  yes
 edge :  8121
		 prediction :  no
 edge :  8278
		 prediction :  yes
 edge :  1850
		 prediction :  no
 edge :  1684
		 prediction :  yes
 edge :  -396
		 prediction :  no
 edge :  2235
		 prediction :  yes
 edge :  1544
		 prediction :  no
 edge :  5073
		 prediction :  yes
 edge :  3296
		 prediction :  no
 edge :  116
		 prediction :  no
 edge :  2573
		 prediction :  yes
 edge :  -639
		 prediction :  yes
 edge :  2979
		 prediction :  yes
 edge :  5914
		 prediction :  no
 edge :  3953
		 prediction :  no
 edge :  1792
		 prediction :  yes
 edge :  1305
		 prediction :  no
 edge :  1496
		 prediction :  no
 edge :  7620
		 prediction :  yes
 edge :  2984
		 prediction :  yes
 edge :  2459
		 prediction :  no
 edge :  3119


		 prediction :  yes
 edge :  1196
		 prediction :  yes
 edge :  9713
		 prediction :  no
 edge :  -168
		 prediction :  yes
 edge :  932
		 prediction :  yes
 edge :  3243
		 prediction :  yes
 edge :  2095
		 prediction :  no
 edge :  896
		 prediction :  no
 edge :  1525
		 prediction :  no
 edge :  2524
		 prediction :  yes
 edge :  834
		 prediction :  yes
 edge :  3982
		 prediction :  yes
 edge :  -324
		 prediction :  yes
 edge :  -558
		 prediction :  no
 edge :  7918
		 prediction :  yes
 edge :  638
		 prediction :  yes
 edge :  5355
		 prediction :  yes
 edge :  5828
		 prediction :  no
 edge :  1918
		 prediction :  no
 edge :  8491
		 prediction :  yes
 edge :  3768
		 prediction :  no
 edge :  3534
		 prediction :  no
 edge :  -155
		 prediction :  no
 edge :  -69
		 prediction :  yes
 edge :  10635
		 prediction :  no
 edge :  3792
		 prediction :  yes
 edge :  2369
		 prediction :  no
 edge :  9851
		 prediction :  no
 edge :  2766
		 prediction :  no
 edge :  832
		 p